<a href="https://colab.research.google.com/github/7ev3r/Midterm_assignment_v2/blob/main/Train_ML_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366475 sha256=7a9e7915d4554501d62e5fd21ce50cea02af7fd3b46bd3672e25a56947c72c42
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [80]:
import pandas as pd
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np
import time

df = pd.read_csv('Clean_whiskey_data.csv')
df = df.drop('Unnamed: 0', axis = 1)
df.shape

(33304, 6)

In [81]:
pd.options.display.max_colwidth = 10000
df.sample(n=3)

,whiskey,rating,url,User_ID,Whiskey_ID,price_euros
2555,ben-nevis-2004-se,89,https://www.whiskybase.com/whiskies/whisky/224323/ben-nevis-2004-se,2469,240,178.81
12161,glen-garioch-2011-tcah,86,https://www.whiskybase.com/whiskies/whisky/214549/glen-garioch-2011-tcah,235,938,49.90
13778,glenallachie-2008,87,https://www.whiskybase.com/whiskies/whisky/223062/glenallachie-2008,729,1029,129.99


In [82]:
df['whiskey'] = df['whiskey'].str.replace('-', ' ').str.title()
rec_df = df.drop(columns = ['url', 'price_euros'])
rec_df = rec_df[['User_ID', 'Whiskey_ID', 'rating']]
rec_match = df[['url', 'whiskey', 'Whiskey_ID', 'price_euros']].drop_duplicates('Whiskey_ID')
rec_match.head()

,url,whiskey,Whiskey_ID,price_euros
0,https://www.whiskybase.com/whiskies/whisky/218272/1770-2015-glasgow-single-malt,1770 2015 Glasgow Single Malt,0,85.16
6,https://www.whiskybase.com/whiskies/whisky/220758/1770-2016-glasgow-single-malt,1770 2016 Glasgow Single Malt,1,89.89
12,https://www.whiskybase.com/whiskies/whisky/205065/1770-2017,1770 2017,2,159.97
20,https://www.whiskybase.com/whiskies/whisky/217751/1770-2018,1770 2018,3,75.56
27,https://www.whiskybase.com/whiskies/whisky/220759/1770-2018-glasgow-single-malt,1770 2018 Glasgow Single Malt,4,83.32


In [83]:
rec_df.User_ID.max()

2518

Checking RMSE 

In [89]:
from surprise import Reader, Dataset
reader = Reader(rating_scale = (1,100))
data = Dataset.load_from_df(rec_df,reader)
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=0.25, random_state = 100)
svd = SVD()
svd.fit(trainset)
preds = svd.test(testset)
accuracy.rmse(preds)

RMSE: 2.6836


2.683607900966367

In [51]:
def test_pal(uid, iid, actual):
    pred = svd.predict(uid, iid, actual, verbose=True)
    match = rec_match.loc[rec_match['Whiskey_ID'] == iid]
    return match

test_pal(601,0, 90)

user: 601        item: 0          r_ui = 90.00   est = 87.18   {'was_impossible': False}


,url,whiskey,Whiskey_ID,price_euros
0,https://www.whiskybase.com/whiskies/whisky/218272/1770-2015-glasgow-single-malt,1770 2015 Glasgow Single Malt,0,85.16


In [78]:
def find_booze(user_ratings, num_recs, priceEUR):
    new_ratings_df = rec_match.append(user_ratings,ignore_index=True)
    new_ratings_df = new_ratings_df[new_ratings_df['price_euros'] < priceEUR]
    new_ratings_df = new_ratings_df.drop(columns=['url', 'whiskey', 'price_euros'])

    #load in new df
    new_data = Dataset.load_from_df(new_ratings_df,reader)
    #new svd object
    svd_new = SVD()
    #refit the model
    svd_new.fit(new_data.build_full_trainset())

    # predictions
    list_of_whiskies = []
    for w_id in new_ratings_df['Whiskey_ID'].unique():
        list_of_whiskies.append((w_id, svd_new.predict(new_ratings_df['User_ID'].max(),w_id)[3]))

    # sort results from highest to lowest rated
    sorted_list = sorted(list_of_whiskies, key=lambda x:x[1],reverse=True)

    rec_num = 1
    for i in sorted_list[:num_recs]:
        found_booze = rec_match[rec_match['Whiskey_ID'] == i[0]]
        print('Offer number:', rec_num)
        print('Whiskey: ' + found_booze.values[0][1])
        print('Average Price: ' + str(found_booze.values[0][-1]))
        print('URL: ' + found_booze.values[0][0])
        print('\n')
        rec_num +=1

def BoozePal(df, num):
    userID = rec_df.User_ID.max()+1
    num_recs = input('How many offers you need ? Enter a number from 1 to 10:\n')
    
    while int(num_recs) > 10:
        num_recs = input('You entered a number over 10. Please enter a number from 1 to 10 to continue. \n')
    priceEUR = input('Enter your budget for a bottle. EUR ')
    rating_list = []
    
    while num > 0:
        whiskey = df[df['price_euros'] < int(priceEUR)].sample(1)
        print('\nPlease rate the following {} whiskies. \n'.format(num))
        print('Whiskey: ' + whiskey.values[0][0])
        print('Price: ' + str(whiskey.values[0][-1]))
        print('URL: ' + whiskey.values[0][2])
        rating = input('Rate this whiskey from 1 to 100, press n if you do not know it. :\n')
            
        if rating == 'n':
            continue
        if int(rating) > 100:
            print('Rating must be below 100!')
            continue
        else:
            rating_one_whiskey = {'User_ID':userID,'Whiskey_ID': whiskey['Whiskey_ID'].values[0],'rating': rating}
            rating_list.append(rating_one_whiskey) 
            num -= 1
        time.sleep(1.5)
    print('\n'+'BoozePal offers : '+'\n')
    
    time.sleep(1.5)
    find_booze(rating_list, int(num_recs), int(priceEUR))

In [75]:
df.head()

,whiskey,rating,url,User_ID,Whiskey_ID,price_euros
0,1770 2015 Glasgow Single Malt,92,https://www.whiskybase.com/whiskies/whisky/218272/1770-2015-glasgow-single-malt,1131,0,85.16
1,1770 2015 Glasgow Single Malt,89,https://www.whiskybase.com/whiskies/whisky/218272/1770-2015-glasgow-single-malt,1017,0,85.16
2,1770 2015 Glasgow Single Malt,88,https://www.whiskybase.com/whiskies/whisky/218272/1770-2015-glasgow-single-malt,1333,0,85.16
3,1770 2015 Glasgow Single Malt,87,https://www.whiskybase.com/whiskies/whisky/218272/1770-2015-glasgow-single-malt,1133,0,85.16
4,1770 2015 Glasgow Single Malt,84,https://www.whiskybase.com/whiskies/whisky/218272/1770-2015-glasgow-single-malt,1108,0,85.16


In [90]:
BoozePal(df, 5)

How many to offer ? Enter a number from 1 to 10:
5
Enter your budget for a bottle. EUR 80

Please rate the following 5 whiskies. 

Whiskey: Islay Blended Malt Scotch Whisky Sherry Cask Eld
Price: 68.2
URL: https://www.whiskybase.com/whiskies/whisky/212288/islay-blended-malt-scotch-whisky-sherry-cask-eld
Rate this whiskey from 1 to 100, press n if you do not know it. :
60

Please rate the following 4 whiskies. 

Whiskey: 1770 Glasgow Single Malt
Price: 46.85
URL: https://www.whiskybase.com/whiskies/whisky/211644/1770-glasgow-single-malt
Rate this whiskey from 1 to 100, press n if you do not know it. :
20

Please rate the following 3 whiskies. 

Whiskey: Glenlossie 2009 Sv
Price: 58.93
URL: https://www.whiskybase.com/whiskies/whisky/203803/glenlossie-2009-sv
Rate this whiskey from 1 to 100, press n if you do not know it. :
50

Please rate the following 2 whiskies. 

Whiskey: Glen Garioch 2011 Brd
Price: 73.25
URL: https://www.whiskybase.com/whiskies/whisky/218228/glen-garioch-2011-brd
Ra